## 2.4 Model Selection 소개

### 학습/테스트 데이터 셋 분리 - train_test_split()

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris = load_iris()
dt_clf = DecisionTreeClassifier()
train_data = iris.data
train_label = iris.target
dt_clf.fit(train_data, train_label)

# 학습 데이터 셋으로 예측 수행
# 학습 데이터로 예측을 하기 때문에 예측 정확도는 100%임, 별도의 테스트 데이터를 만들어야함
pred = dt_clf.predict(train_data)
print('예측 정확도: ', accuracy_score(train_label, pred))

예측 정확도:  1.0


In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

dt_clf = DecisionTreeClassifier()
iris_data = load_iris()

# test_size: 학습 데이터와 테스트 데이터의 비율 (ex 0.3 이라면 학습 데이터 70%, 테스트 데이터 30%)
# random_state: 랜덤 난수 설정
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, test_size=0.3, random_state=121)

In [3]:
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

예측 정확도: 0.9556


In [4]:
import pandas as pd

iris_df = pd.DataFrame(iris_data.data, columns = iris_data.feature_names)
iris_df['target'] = iris_data.target

iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [5]:
# target을 제외한 data set
ftr_df = iris_df.iloc[:, :-1]

# target data set
tgt_df = iris_df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(ftr_df, tgt_df, test_size=0.3, random_state=121)

In [6]:
print(type(X_train), type(X_test), type(y_train), type(y_test))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


In [7]:
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

예측 정확도: 0.9556


### 교차 검증
* K Fold
    * K번 학습과 검증 평가 반복을 수행하여 알고리즘을 평가

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=156)

# 5개의 Fold set로 분리하는 KFold 객체와 Fold set별 정확도를 담을 리스트 객체 생성
kfold = KFold(n_splits = 5)
cv_accuracy = []

print('붓꽃 데이터 세트 크기: ', features.shape[0])

붓꽃 데이터 세트 크기:  150


In [9]:
n_iter = 0

# KFold 객체의 split()은 Fold별 학습용, 검증용 테스트의 로우 인덱스를 array로 반환
for train_index, test_index in kfold.split(features):
    
    # kfold.split()으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    # 학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    n_iter += 1
    
    # 반복 시 마다 정확도 측정
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('{0} 교차 검증 정확도: {1}\n학습 데이터 크기: {2}\n검증 데이터 크기: {3}'.format(n_iter, accuracy, train_size, test_size))
    print('{0} 검증 세트 인덱스: {1}\n'.format(n_iter, test_index))
    
    cv_accuracy.append(accuracy)
    
# 개별 정확도를 합하여 평균 정확도 계산
print('## 평균 검증 정확도: ', np.mean(cv_accuracy))

1 교차 검증 정확도: 1.0
학습 데이터 크기: 120
검증 데이터 크기: 30
1 검증 세트 인덱스: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

2 교차 검증 정확도: 0.9667
학습 데이터 크기: 120
검증 데이터 크기: 30
2 검증 세트 인덱스: [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

3 교차 검증 정확도: 0.8667
학습 데이터 크기: 120
검증 데이터 크기: 30
3 검증 세트 인덱스: [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

4 교차 검증 정확도: 0.9333
학습 데이터 크기: 120
검증 데이터 크기: 30
4 검증 세트 인덱스: [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

5 교차 검증 정확도: 0.7333
학습 데이터 크기: 120
검증 데이터 크기: 30
5 검증 세트 인덱스: [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도:  0.9


* Stratified K Fold
    * 불균형한 분포도를 가진 레이블 데이터 집합을 위한 K 폴드 방식
    * 학습, 검증 데이터 세트가 가지는 레이블 분포도가 유사하도록 검증 데이터를 추출
    * 분류에서 반드시 사용
    
ex) 신용카드의 사기 발병률이 1%라고 가정했을 때, 10000건의 결제 내역에서 100건이 사기가 된다. 이 때 사기 발병률을 계산하는 알고리즘에서 특정 몇개의 학습데이터에 사기 발생 데이터가 존재하지 않을 확률이 크기 때문에 사기 발생과 관련된 패턴을 학습하는데 어려움이 있다. 때문에 레이블의 분포도가 유사하도록 검증 데이터를 추출한다.

In [10]:
import pandas as pd

iris = load_iris()

iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label'] = iris.target
iris_df['label'].value_counts()

2    50
1    50
0    50
Name: label, dtype: int64

In [11]:
kfold = KFold(n_splits=3)

n_iter = 0

# 검증 하고자하는 레이블의 학습이 전혀 안되므로 잘못된 Case
for train_index, test_index in kfold.split(iris_df):
    n_iter += 1
    
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포\n', label_train.value_counts())
    print('검증 레이블 데이터 분포\n', label_test.value_counts())    

## 교차 검증: 1
학습 레이블 데이터 분포
 2    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포
 0    50
Name: label, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포
 2    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포
 1    50
Name: label, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포
 1    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포
 2    50
Name: label, dtype: int64


In [12]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n_iter = 0

# StratifiedKFold.split은 결정값도 인자로 넣어야함
for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_iter += 1
    
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포\n', label_train.value_counts())
    print('검증 레이블 데이터 분포\n', label_test.value_counts())   

## 교차 검증: 1
학습 레이블 데이터 분포
 2    34
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포
 1    17
0    17
2    16
Name: label, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포
 1    34
2    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포
 2    17
0    17
1    16
Name: label, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포
 0    34
2    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포
 2    17
1    17
0    16
Name: label, dtype: int64


In [13]:
dt_clf = DecisionTreeClassifier(random_state=156)

skfold = StratifiedKFold(n_splits=3)
n_iter = 0
cv_accuracy = []

for train_index, test_index in skfold.split(features, label):
    
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    n_iter += 1
    
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('{0} 교차 검증 정확도: {1}\n학습 데이터 크기: {2}\n검증 데이터 크기: {3}'.format(n_iter, accuracy, train_size, test_size))
    print('{0} 검증 세트 인덱스: {1}\n'.format(n_iter, test_index))
    
    cv_accuracy.append(accuracy)

print('## 교차 검증별 정확도: ', np.round(cv_accuracy, 4))
print('## 평균 검증 정확도: ', np.mean(cv_accuracy))

1 교차 검증 정확도: 0.98
학습 데이터 크기: 100
검증 데이터 크기: 50
1 검증 세트 인덱스: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

2 교차 검증 정확도: 0.94
학습 데이터 크기: 100
검증 데이터 크기: 50
2 검증 세트 인덱스: [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

3 교차 검증 정확도: 0.98
학습 데이터 크기: 100
검증 데이터 크기: 50
3 검증 세트 인덱스: [ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도:  [0.98 0.94 0.98]
## 평균 검증 정확도:  0.9666666666666667


* cross_val_score()
    * 위의 반복 학습 및 검증을 간단하게 수행

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris
import numpy as np

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
label = iris_data.target

# 성능 지표는 정확도(accuracy), 교차 검증 세트 = 3
scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=3)
print('교차 검증별 정확도: ', np.round(scores, 4))
print('평균 검증 정확도: ', np.round(np.mean(scores), 4))

교차 검증별 정확도:  [0.98 0.94 0.98]
평균 검증 정확도:  0.9667


* GridSearchCV
    * 하이퍼 파라미터를 순차적으로 입력하며 최적의 파라미터를 도출할 수 있는 방안을 제공

In [15]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

# 데이터를 로딩하고 학습데이타와 테스트 데이터 분리
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, 
                                                    test_size=0.2, random_state=121)
dtree = DecisionTreeClassifier()

# parameter들을 dictionary 형태로 설정
# max_depth: tree의 깊이
# min_samples_split: 몇 개의 데이터가 있어야 tree를 분할할 것인가
# 파라미터 순차 적용 횟수 6번 (1, 2), (1, 3), (2, 2), (2, 3), (3, 2), (3, 3)
parameters = {'max_depth': [1, 2, 3], 'min_samples_split': [2, 3]}

In [16]:
import pandas as pd

# param_grid의 하이퍼 파라미터들을 3개의 train, test set fold로 나눠 테스트 수행
# refit = True가 defalut임, True이면 가장 좋은 파라미터 설정으로 재학습 시킴
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True, return_train_score=True)

# 붓꽃 Train 데이터로 param_grid의 하이퍼 파라미터들을 순차적으로 학습/검증
grid_dtree.fit(X_train, y_train)

# GridSearchCV 결과는 cv_results_ 라는 딕셔너리로 저장됨. 이를 DataFrame으로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [17]:
grid_dtree.cv_results_

{'mean_fit_time': array([0.00033967, 0.00033315, 0.00100009, 0.00066686, 0.00066622,
        0.00099993]),
 'std_fit_time': array([4.80361681e-04, 4.71145571e-04, 4.89903609e-07, 4.71538951e-04,
        4.71093809e-04, 1.66324373e-06]),
 'mean_score_time': array([0., 0., 0., 0., 0., 0.]),
 'std_score_time': array([0., 0., 0., 0., 0., 0.]),
 'param_max_depth': masked_array(data=[1, 1, 2, 2, 3, 3],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_split': masked_array(data=[2, 3, 2, 3, 2, 3],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 1, 'min_samples_split': 2},
  {'max_depth': 1, 'min_samples_split': 3},
  {'max_depth': 2, 'min_samples_split': 2},
  {'max_depth': 2, 'min_samples_split': 3},
  {'max_depth': 3, 'min_samples_split': 2},
  {'max_depth': 3, 'min_samples_split': 3}],
 'split0_test_score': array([

In [18]:
print('GridSearchCV 최적 파라미터: ', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도: {0: .4f}'.format(grid_dtree.best_score_))

# refit = True로 설정된 GridSearchCV 객체가 fit() 수행 시 학습이 완료된 Estimator를 내포하고 있어 예측 가능
# GridSearchCV와 테스트 데이터 세트 정확도가 다른 이유
# 위 소스 코드에서 test_size = 0.2로 학습 데이터 세트 (80%), 테스트 데이터 세트 (20%)가 분리되어 있다.
# GridSearchCV의 cv = 3으로 주어진 X_train, y_train으로 3개의 Fold로 학습/검증 데이터를 분할한다.
# 즉, X_train을 쪼개면서 학습하고 예측한 최고의 정확도를 grid_dtree.best_score_가 갖게 된다.
# 하지만, 이는 테스트 데이터 X_test로 predict() 된게 아니므로 X_test의 predict()와 결과가 다르다.
pred = grid_dtree.predict(X_test)
print('테스트 데이터 세트 정확도: {0: .4f}'.format(accuracy_score(y_test, pred)))

GridSearchCV 최적 파라미터:  {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도:  0.9750
테스트 데이터 세트 정확도:  0.9667


In [19]:
# GridSearchCV의 refit으로 이미 학습이 된 estimator 반환
estimator = grid_dtree.best_estimator_

# GridSearchCV의 best_estimator_는 이미 최적 하이퍼 파라미터로 학습이 됨
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

테스트 데이터 세트 정확도: 0.9667
